In [ ]:
import os.path

def path(name):
    root = os.path.dirname(os.path.realpath(__file__))
    return os.path.join(root, 'models', name)

# Models
FACE_DETECTION_MODEL = path('ssd_mobilenet_v2_face_quant_postprocess_edgetpu.tflite')
OBJECT_DETECTION_MODEL = path('ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite')
CLASSIFICATION_MODEL = path('tf2_mobilenet_v2_1.0_224_ptq_edgetpu.tflite')
CLASSIFICATION_IMPRINTING_MODEL = path('mobilenet_v1_1.0_224_l2norm_quant_edgetpu.tflite')
MOVENET_MODEL = path('movenet_single_pose_lightning_ptq_edgetpu.tflite')

# Labels
CLASSIFICATION_LABELS = path('imagenet_labels.txt')
OBJECT_DETECTION_LABELS = path('coco_labels.txt')

In [ ]:

from pycoral.utils import edgetpu
from pycoral.adapters import detect
import cv2
import numpy as np
import tflite_runtime.interpreter as tflite

from pycoral.adapters import common

In [ ]:
CORAL_COLOR = (86, 104, 237)

model = "./example/models/tf2_mobilenet_v2_1.0_224_ptq_edgetpu.tflite"

interpreter = edgetpu.make_interpreter(model)
interpreter.allocate_tensors()


def get_objects(frame, threshold=0.01):

    height, width, _ = frame.shape
    _, scale = common.set_resized_input(interpreter, (width, height),
                                        lambda size: cv2.resize(frame, size, fx=0, fy=0, interpolation=cv2.INTER_CUBIC))
    interpreter.invoke()
    return detect.get_objects(interpreter, threshold, scale)


def draw_objects(frame, objs, labels=None, color=CORAL_COLOR, thickness=5):

    for obj in objs:
        bbox = obj.bbox
        cv2.rectangle(frame, (bbox.xmin, bbox.ymin), (bbox.xmax, bbox.ymax),
                      color, thickness)
        if labels: 
            cv2.putText(frame, labels.get(obj.id), (bbox.xmin + thickness, bbox.ymax - thickness),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=CORAL_COLOR, thickness=2)

            

In [ ]:
class Detector:
    """Performs inferencing with an object detection model.

    Args:
      model (str): Path to a ``.tflite`` file (compiled for the Edge TPU).
        Must be an SSD model.
    """

    def __init__(self, model):
        self.interpreter = edgetpu.make_interpreter(model)
        self.interpreter.allocate_tensors()

    def get_objects(self, frame, threshold=0.01):

        height, width, _ = frame.shape
        _, scale = common.set_resized_input(self.interpreter, (width, height),
                                            lambda size: cv2.resize(frame, size,
                                                                    fx=0, fy=0,
                                                                    interpolation=cv2.INTER_CUBIC))
        self.interpreter.invoke()
        return detect.get_objects(self.interpreter, threshold, scale)
    

    

In [ ]:

from aiymakerkit import vision
import models

detector = vision.Detector(models.FACE_DETECTION_MODEL)

for frame in vision.get_frames():
    faces = detector.get_objects(frame, threshold=0.1)
    vision.draw_objects(frame, faces)